In [ ]:
# Install dependencies
!pip install diffusers transformers accelerate safetensors flask flask-cors flask-ngrok

In [ ]:
# Imports
from flask import Flask, request, send_file
from flask_cors import CORS
from flask_ngrok import run_with_ngrok
from diffusers import StableDiffusionPipeline
import torch
import uuid
import os

In [ ]:
# Create Flask app
app = Flask(__name__)
CORS(app)
run_with_ngrok(app)  # Must be after app definition

In [ ]:
# Load Waifu Diffusion model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
pipe = StableDiffusionPipeline.from_pretrained(
    'hakurei/waifu-diffusion',
    torch_dtype=torch.float16
)
pipe = pipe.to(device)

# Output folder
output_dir = 'outputs'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Define API endpoint
@app.route('/generate', methods=['GET'])
def generate():
    prompt = request.args.get('prompt')
    if not prompt:
        return '❌ Error: prompt query required', 400

    output_path = os.path.join(output_dir, f'{uuid.uuid4()}.png')
    image = pipe(prompt, guidance_scale=7.5).images[0]
    image.save(output_path)

    return send_file(output_path, mimetype='image/png')

In [ ]:
# Run Flask app with ngrok
app.run()